In [1]:
%cd ../..

/home/eli/AnacondaProjects/epych


In [2]:
%env DASK_LOGGING__DISTRIBUTED=CRITICAL
%env SPYTMPDIR=/mnt/data/tmp_storage
%env SPYLOGLEVEL=CRITICAL
%env SPYPARLOGLEVEL=CRITICAL

env: DASK_LOGGING__DISTRIBUTED=CRITICAL
env: SPYTMPDIR=/mnt/data/tmp_storage
env: SPYLOGLEVEL=CRITICAL
env: SPYPARLOGLEVEL=CRITICAL


In [3]:
import collections
import functools
import logging
import numpy as np
import os

import epych
from epych.statistics import alignment

[striatum:1610881] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.striatum.1000/jf.0/4002021376/shared_mem_cuda_pool.striatum could be created.
[striatum:1610881] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


In [4]:
%matplotlib inline

In [5]:
logging.basicConfig(level=logging.INFO)

In [6]:
CONDITIONS = ["lonaive", "go_gloexp", "go_seqctl", "lo_gloexp", "lo_rndctl", "igo_seqctl"]

In [7]:
def samplings(cond):
    sampling = epych.recording.Sampling.unpickle("/mnt/data/DRAFT/000253/grandcat_%s" % cond)
    logging.info("Loaded LFPs for condition %s, baseline from (%f, %f)" % (cond, sampling.trials["trial_start"].mean(), sampling.trials["trial_start"].mean() + 0.5))
    yield sampling.smap(lambda sig: sig.downsample(16))

In [8]:
def initialize_spectrum(key, signal, path=None):
    area = os.path.commonprefix([loc for loc in signal.channels.location])
    return epych.statistics.spectrum.Spectrogram(signal.df, signal.channels, signal.f0, chunk_trials=50, taper="hann", path=path + "/" + key)

In [9]:
for cond in CONDITIONS:
    logging.info("Calculating LFP spectrograms across condition %s" % cond)
    cond_path = "/mnt/data/000253/grand_spectrogram_downsample16_%s" % cond
    if os.path.exists(cond_path):
        continue
    summary = epych.statistic.Summary(alignment.location_prefix, functools.partial(initialize_spectrum, path=cond_path))
    summary.calculate(samplings(cond))
    summary.pickle(cond_path)
    del summary

INFO:root:Calculating LFP spectrograms across condition lonaive
INFO:root:Loaded LFPs for condition lonaive, baseline from (-3.002524, -2.502524)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [06:43<00:00, 26.87s/it]
INFO:root:Calculating LFP spectrograms across condition go_gloexp
INFO:root:Loaded LFPs for condition go_gloexp, baseline from (-3.002630, -2.502630)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████